# Gold Layer: Data Aggregation for Reporting
This notebook aggregates data from the Silver Layer in MySQL and stores the results in a Gold Layer table for reporting in Power BI.

In [ ]:

import mysql.connector
import pandas as pd

In [ ]:
# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='aashrits',
    password='Aashu@2003',
    database='silver_db'
)
cursor = conn.cursor()

In [ ]:

cursor.execute("CREATE DATABASE IF NOT EXISTS gold_db")
cursor.execute("USE gold_db")
cursor.execute("""
CREATE TABLE IF NOT EXISTS sales_gold (
    category VARCHAR(255),
    total_sales_value FLOAT,
    total_quantity INT,
    avg_price FLOAT,
    store_name VARCHAR(255),
    location VARCHAR(255),
    report_date DATE
)""")

In [ ]:

cursor.execute("""
INSERT INTO gold_db.sales_gold (category, total_sales_value, total_quantity, avg_price)
SELECT p.category, 
       SUM(s.total_sales_value), 
       SUM(s.quantity), 
       AVG(s.price)
FROM silver_db.sales_silver s
JOIN silver_db.product_silver p ON s.product_id = p.product_id
GROUP BY p.category
""")

In [ ]:

cursor.execute("""
INSERT INTO gold_db.sales_gold (store_name, location, total_sales_value)
SELECT st.store_name, st.location, SUM(s.total_sales_value)
FROM silver_db.sales_silver s
JOIN silver_db.store_silver st ON s.store_id = st.store_id
GROUP BY st.store_name, st.location
""")

In [ ]:

cursor.execute("""
INSERT INTO gold_db.sales_gold (report_date, total_sales_value)
SELECT s.sale_date AS report_date, SUM(s.total_sales_value)
FROM silver_db.sales_silver s
GROUP BY s.sale_date
""")
conn.commit()

In [ ]:

cursor.close()
conn.close()